In [1]:
import tensorflow as tf

### 2.1 自动求梯度
- 在深度学习中，我们经常需要对函数求梯度（gradient）。本节将介绍如何使用tensorflow2.0提供的GradientTape来自动求梯度。

2.3.1 简单示例  y = 2 * x * x 求梯度

In [12]:
x = tf.reshape(tf.Variable(range(4),dtype=tf.float32),(4,1))
with tf.GradientTape() as t:
    t.watch(x)
    y = 2 * tf.matmul(tf.transpose(x),x) # 28
dy_dx = t.gradient(y,x)
dy_dx

<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[ 0.],
       [ 4.],
       [ 8.],
       [12.]], dtype=float32)>

GradientTape也可以嵌套多层用来计算高阶导数

In [14]:
x = tf.reshape(tf.Variable(range(4),dtype=tf.float32),(4,1))
with tf.GradientTape() as t:
    t.watch(x)
    with tf.GradientTape() as tt:
        tt.watch(x)
        y = 2 * tf.matmul(tf.transpose(x),x) # 28
    dy_dx = tt.gradient(y,x)
dy2_dx2 = t.gradient(dy_dx,x)
print(dy_dx,dy2_dx2)

tf.Tensor(
[[ 0.]
 [ 4.]
 [ 8.]
 [12.]], shape=(4, 1), dtype=float32) tf.Tensor(
[[4.]
 [4.]
 [4.]
 [4.]], shape=(4, 1), dtype=float32)


另外，默认情况下GradientTape的资源在调用gradient函数后就被释放，再次调用就无法计算了。所以如果需要多次计算梯度，需要开启persistent=True属性，

In [17]:
x = tf.reshape(tf.Variable(range(4),dtype=tf.float32),(4,1))
with tf.GradientTape(persistent=True) as t:
    t.watch(x)
    y = 2 * tf.matmul(tf.transpose(x),x) # 28
    z = 2 * y
dz_dx = t.gradient(z,x)
dy_dx = t.gradient(y,x)
print(dy_dx,dz_dx)
del t  # 删除这个上下文胶带

tf.Tensor(
[[ 0.]
 [ 4.]
 [ 8.]
 [12.]], shape=(4, 1), dtype=float32) tf.Tensor(
[[ 0.]
 [ 8.]
 [16.]
 [24.]], shape=(4, 1), dtype=float32)


一般在网络中使用时，不需要显式调用watch函数，使用默认设置，GradientTape会监控可训练变量，例如：

In [ ]:
with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
gradients = tape.gradient(loss, model.trainable_variables)
#这样即可计算出所有可训练变量的梯度，然后进行下一步的更新

### 查阅文档

受篇幅所限，本书无法对所有用到的tensorflow2.0函数和类一一详细介绍。读者可以查阅相关文档来做更深入的了解。

#### search for functions and classes
- 当我们想知道一个模块里面提供了哪些可以调用的函数和类的时候，可以使用dir函数。下面我们打印dtypes和random模块中所有的成员或属性。

In [18]:
dir(tf.dtypes)

['DType',
 'QUANTIZED_DTYPES',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_sys',
 'as_dtype',
 'bfloat16',
 'bool',
 'cast',
 'complex',
 'complex128',
 'complex64',
 'double',
 'float16',
 'float32',
 'float64',
 'half',
 'int16',
 'int32',
 'int64',
 'int8',
 'qint16',
 'qint32',
 'qint8',
 'quint16',
 'quint8',
 'resource',
 'saturate_cast',
 'string',
 'uint16',
 'uint32',
 'uint64',
 'uint8',
 'variant']

In [19]:
dir(tf.random)

['Algorithm',
 'Generator',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_sys',
 'all_candidate_sampler',
 'categorical',
 'create_rng_state',
 'experimental',
 'fixed_unigram_candidate_sampler',
 'gamma',
 'get_global_generator',
 'learned_unigram_candidate_sampler',
 'log_uniform_candidate_sampler',
 'normal',
 'poisson',
 'set_global_generator',
 'set_seed',
 'shuffle',
 'stateless_binomial',
 'stateless_categorical',
 'stateless_gamma',
 'stateless_normal',
 'stateless_parameterized_truncated_normal',
 'stateless_poisson',
 'stateless_truncated_normal',
 'stateless_uniform',
 'truncated_normal',
 'uniform',
 'uniform_candidate_sampler']

通常我们可以忽略掉由__开头和结尾的函数（Python的特别对象）或者由_开头的函数（一般为内部函数）。通过其余成员的名字我们大致猜测出这个模块提供了各种随机数的生成方法，包括从均匀分布采样（uniform）、从正态分布采样（normal）、从泊松分布采样（poisson）等。